## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import ast
from collections import Counter
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import re

## Cleaning

In [2]:
with open('games.csv', encoding='utf-8') as f:
    s = f.read()

with open('games_Fixed.csv', 'w', encoding='utf-8') as f:
    f.write(s.replace('DiscountDLC', 'Discount,DLC', 1))

In [3]:
df = pd.read_csv('games_Fixed.csv')

df.head(5)

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,Discount,DLC count,About the game,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,0,Galactic Bowling is an exaggerated and stylize...,...,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,0,THE LAW!! Looks to be a showdown atop a train....,...,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,0,Jolt Project: The army now has a new robotics ...,...,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,...,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,0,ABOUT THE GAME Play as a hacker who has arrang...,...,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [4]:
print("Info:")
print(df.info())

print("Missing Values:")
missing_values = df.isnull().sum()
display(missing_values[missing_values > 0].sort_values(ascending=False))

print("Basic Statistics:")
display(df.describe())

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97410 entries, 0 to 97409
Data columns (total 40 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   AppID                       97410 non-null  int64  
 1   Name                        97404 non-null  object 
 2   Release date                97410 non-null  object 
 3   Estimated owners            97410 non-null  object 
 4   Peak CCU                    97410 non-null  int64  
 5   Required age                97410 non-null  int64  
 6   Price                       97410 non-null  float64
 7   Discount                    97410 non-null  int64  
 8   DLC count                   97410 non-null  int64  
 9   About the game              92540 non-null  object 
 10  Supported languages         97410 non-null  object 
 11  Full audio languages        97410 non-null  object 
 12  Reviews                     10125 non-null  object 
 13  Header image             

Score rank        97366
Metacritic url    93457
Reviews           87285
Notes             81936
Website           54673
Support url       51502
Tags              29763
Support email     16032
Movies             7891
Categories         5913
Publishers         5136
Developers         4876
About the game     4870
Genres             4841
Screenshots        2895
Name                  6
dtype: int64

Basic Statistics:


,AppID,Peak CCU,Required age,Price,Discount,DLC count,Metacritic score,User score,Positive,Negative,Score rank,Achievements,Recommendations,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks
count,9.741000e+04,9.741000e+04,97410.000000,97410.000000,97410.0,97410.000000,97410.000000,97410.000000,9.741000e+04,97410.000000,44.000000,97410.000000,9.741000e+04,97410.000000,97410.000000,97410.000000,97410.000000
mean,1.524636e+06,1.627283e+02,0.282497,7.086262,0.0,0.493081,2.959193,0.034791,8.489366e+02,141.065989,98.909091,18.562622,6.905128e+02,91.797937,9.621271,81.852459,10.340417
std,8.001310e+05,7.687116e+03,2.144294,12.387782,0.0,12.832834,14.545462,1.674105,2.287011e+04,4278.019743,0.857747,160.413540,1.681717e+04,1068.494928,177.190210,1412.496536,192.711603
min,1.000000e+01,0.000000e+00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000,97.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,8.477925e+05,0.000000e+00,0.000000,0.990000,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000,98.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
50%,1.489990e+06,0.000000e+00,0.000000,4.190000,0.0,0.000000,0.000000,0.000000,5.000000e+00,1.000000,99.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
75%,2.180648e+06,1.000000e+00,0.000000,9.990000,0.0,0.000000,0.000000,0.000000,3.500000e+01,10.000000,100.000000,17.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
max,3.199580e+06,1.284268e+06,21.000000,999.980000,0.0,2366.000000,97.000000,100.000000,5.764420e+06,895978.000000,100.000000,9821.000000,3.441592e+06,145727.000000,19159.000000,208473.000000,19159.000000


In [5]:
columns_to_drop = [
    'Reviews', 'Metacritic url', 'Score rank', 'Notes', # High NaNs
    'Website', 'Support url', 'Support email', # URLs/Emails
    'Header image', 'Screenshots', 'Movies', # Media links
    'AppID', # Identifiers
    'Discount', # No variance (always 0)
    'User score' # Unreliable (mostly 0)
]

df.drop(columns=columns_to_drop, inplace=True, errors='ignore')
print(f"Dropped {len(columns_to_drop)} columns.")

print(f"Remaining columns: {df.shape[1]}")
print("\nColumns after initial drop:")
print(df.columns.tolist())

print("Missing Values:")
missing_values = df.isnull().sum()
display(missing_values[missing_values > 0].sort_values(ascending=False))

drop_na_columns = [
    'Categories', 'Genres', 'Tags', # Since they are needed for analysis, we will drop rows with NaN values in these columns since can't infer anything from them
    'Publishers', 'Developers', # Also cannot infer anything from them being empty
    "About the game", # We plan to do NLP on this column, so we drop rows as well
]

print(f"Rows before dropping NaN: {df.shape[0]}")
df.dropna(subset=drop_na_columns, inplace=True)

print(f"Dropped {len(drop_na_columns)} columns with NaN values.")
print(f"Rows after dropping NaN: {df.shape[0]}")

print(f"New Shape: {df.shape}")

Dropped 13 columns.
Remaining columns: 27

Columns after initial drop:
['Name', 'Release date', 'Estimated owners', 'Peak CCU', 'Required age', 'Price', 'DLC count', 'About the game', 'Supported languages', 'Full audio languages', 'Windows', 'Mac', 'Linux', 'Metacritic score', 'Positive', 'Negative', 'Achievements', 'Recommendations', 'Average playtime forever', 'Average playtime two weeks', 'Median playtime forever', 'Median playtime two weeks', 'Developers', 'Publishers', 'Categories', 'Genres', 'Tags']
Missing Values:


Tags              29763
Categories         5913
Publishers         5136
Developers         4876
About the game     4870
Genres             4841
Name                  6
dtype: int64

Rows before dropping NaN: 97410
Dropped 6 columns with NaN values.
Rows after dropping NaN: 66272
New Shape: (66272, 27)


In [6]:
df['Release date'] = pd.to_datetime(df['Release date'], errors='coerce') # Converting to datetime for time series analysis

print(f"Earliest release date: {df['Release date'].min()}")

reference_date = df['Release date'].min()
df['Game Age (Days)'] = (df['Release date'] - reference_date).dt.days
median_age = df['Game Age (Days)'].median()

print(f"Number of missing values in 'Release date': {df['Release date'].isnull().sum()}")

print(f"Median game age: {median_age} days")
df.drop(columns=['Release date'], inplace=True, errors='ignore') # No longer needed

print(df['Game Age (Days)'].describe())

Earliest release date: 1997-06-30 00:00:00
Number of missing values in 'Release date': 0
Median game age: 8324.0 days
count    66272.000000
mean      8177.432068
std       1102.109736
min          0.000000
25%       7495.000000
50%       8324.000000
75%       8992.000000
max       9924.000000
Name: Game Age (Days), dtype: float64


In [7]:
def is_indie(genres):
    return 1 if 'Indie' in genres else 0

df['is_indie'] = df['Genres'].apply(is_indie)

print(f"Number of indie games: {df['is_indie'].sum()}")
print(f"Number of non-indie games: {df['is_indie'].count() - df['is_indie'].sum()}")
print(f"Proportion of indie games: {df['is_indie'].mean():.2%}")

Number of indie games: 47847
Number of non-indie games: 18425
Proportion of indie games: 72.20%


In [8]:
print("Ranges of owners")
print(df['Estimated owners'].unique())

ranges = {
    r: i for i, r in enumerate(df['Estimated owners'].unique(), start=1)
}

def convert_owners_to_category(owners):
    return ranges.get(owners, np.nan)

df['Owner range'] = df['Estimated owners'].apply(convert_owners_to_category)
print(df['Owner range'].head())

print(f"Number of unique owner ranges: {df['Owner range'].nunique()}")
df.drop(columns=['Estimated owners'], inplace=True, errors='ignore') # No longer needed

Ranges of owners
['0 - 20000' '50000 - 100000' '20000 - 50000' '200000 - 500000'
 '100000 - 200000' '2000000 - 5000000' '500000 - 1000000'
 '1000000 - 2000000' '20000000 - 50000000' '5000000 - 10000000'
 '10000000 - 20000000' '50000000 - 100000000' '100000000 - 200000000']
0    1
1    1
3    1
4    1
5    2
Name: Owner range, dtype: int64
Number of unique owner ranges: 13


In [9]:
df['Total Reviews'] = df['Positive'] + df['Negative'] # Adding up total positive and negative reviews
df['Review Ratio'] = np.where(
    df['Total Reviews'] > 0,
    df['Positive'] / df['Total Reviews'],
    0.5 # Assigning 0.5 when there are no reviews to avoid division by zero
)

df['Log Total Reviews'] = np.log1p(df['Total Reviews'])
df.drop(columns=['Total Reviews', 'Positive', 'Negative'], inplace=True, errors='ignore') # No longer needed

In [10]:
def parse_language_list(lang_str): # Convert string representation of list to actual list
    if isinstance(lang_str, str) and lang_str.startswith('[') and lang_str.endswith(']'):
        try:
            return ast.literal_eval(lang_str)
        except (ValueError, SyntaxError):
            return []
    return []

df['Languages List'] = df['Supported languages'].apply(parse_language_list)

# Extracting the useful information from the language columns
df['Num Languages'] = df['Languages List'].apply(len) # Extracting number of languages from the list
df['Is English Supported'] = df['Languages List'].apply(lambda x: 'English' in x).astype(int) # Creating a binary column for English support

# Drop the intermediate and original language columns
df.drop(columns=['Supported languages', 'Full audio languages', 'Languages List'], inplace=True, errors='ignore')

display(df[['Num Languages', 'Is English Supported']].head())
df['Num Languages'].describe()

,Num Languages,Is English Supported
0,1,1
1,10,1
3,11,1
4,2,1
5,1,1


count    66272.000000
mean         3.612129
std          4.960837
min          0.000000
25%          1.000000
50%          1.000000
75%          4.000000
max         39.000000
Name: Num Languages, dtype: float64

In [11]:
skewed_cols = [
    'Peak CCU', 'Price', 'Achievements', 'Recommendations',
    'Average playtime forever', 'Average playtime two weeks',
    'Median playtime forever', 'Median playtime two weeks'
]

for col in skewed_cols:
    df[f"Log {col}"] = np.log1p(df[col]) # Log-transforming the skewed columns

log_cols_to_show = [
    'Log Peak CCU', 'Log Price', 'Log Achievements', 'Log Recommendations',
    'Log Average playtime forever', 'Log Average playtime two weeks',
    'Log Median playtime forever', 'Log Median playtime two weeks'
]
display(df[log_cols_to_show].describe())

,Log Peak CCU,Log Price,Log Achievements,Log Recommendations,Log Average playtime forever,Log Average playtime two weeks,Log Median playtime forever,Log Median playtime two weeks
count,66272.000000,66272.000000,66272.000000,66272.000000,66272.000000,66272.000000,66272.000000,66272.000000
mean,0.769264,1.743610,1.678722,1.314865,1.116587,0.148781,1.099117,0.150012
std,1.576763,0.982796,1.640310,2.687193,2.247477,0.887036,2.204946,0.894127
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.095273,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.790091,1.945910,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.693147,2.396986,3.091042,0.000000,0.000000,0.000000,0.000000,0.000000
max,14.065700,6.907755,9.192380,15.051445,11.411468,9.860580,11.825032,9.860580


In [12]:
# Convert Boolean Columns to Integers from Bool
bool_cols = ['Windows', 'Mac', 'Linux']
for col in bool_cols:
    if col in df.columns:
        df[col] = df[col].astype(int)

display(df[bool_cols].head())

,Windows,Mac,Linux
0,1,0,0
1,1,1,0
3,1,1,1
4,1,1,0
5,1,0,0


In [13]:
# print number of unique values in Categories, Genres, Tags
print("Unique values in Categories:")
print(df['Categories'].nunique())
print("Unique values in Genres:")
print(df['Genres'].nunique())
print("Unique values in Tags:")
print(df['Tags'].nunique())

# tags = {tags count total.split(,) for tags count total in df['Tags']}
tags = {}
genres = {}
categories = {}

# Converting all tags, genres, and categories to seperate lists
for index, row in df.iterrows():
    tags_list = row['Tags'].split(',')
    genres_list = row['Genres'].split(',')
    categories_list = row['Categories'].split(',')

    for tag in tags_list:
        tag = tag.strip()
        tags[tag] = tags.get(tag, 0) + 1

    for genre in genres_list:
        genre = genre.strip()
        genres[genre] = genres.get(genre, 0) + 1

    for category in categories_list:
        category = category.strip()
        categories[category] = categories.get(category, 0) + 1

# Sorting the dictionaries by count
tags = dict(sorted(tags.items(), key=lambda item: item[1], reverse=True))
genres = dict(sorted(genres.items(), key=lambda item: item[1], reverse=True))
categories = dict(sorted(categories.items(), key=lambda item: item[1], reverse=True))

# remove Indie from genre and tags since not useful
tags.pop('Indie', None)
genres.pop('Indie', None)

print("Top 100 Tags:")
for tag, count in list(tags.items())[:100]:
    print(f"{tag}: {count}")

print()
print("Top 30 Genres:")
for genre, count in list(genres.items())[:30]:
    print(f"{genre}: {count}")

print()
print("Top 50 Categories:")
for category, count in list(categories.items())[:50]:
    print(f"{category}: {count}")

Unique values in Categories:
5694
Unique values in Genres:
1977
Unique values in Tags:
59748
Top 100 Tags:
Singleplayer: 35664
Action: 30009
Casual: 28736
Adventure: 28219
2D: 18772
Strategy: 14336
Simulation: 14152
RPG: 12651
Puzzle: 12072
Atmospheric: 11997
3D: 11239
Early Access: 9852
Pixel Graphics: 9829
Story Rich: 9680
Colorful: 9617
Exploration: 8816
Cute: 8756
First-Person: 8424
Arcade: 8238
Multiplayer: 8165
Fantasy: 8161
Funny: 7147
Shooter: 6989
Horror: 6782
Retro: 6774
Platformer: 6715
Anime: 6413
Family Friendly: 6411
Sci-fi: 6379
Action-Adventure: 6274
Relaxing: 6250
Female Protagonist: 5608
Difficult: 5384
Third Person: 5131
VR: 5095
Survival: 5066
Top-Down: 4903
Open World: 4881
Stylized: 4676
Controller: 4644
Combat: 4631
Great Soundtrack: 4421
Comedy: 4407
2D Platformer: 4271
Visual Novel: 4268
FPS: 4239
Violent: 4198
Mystery: 4137
Co-op: 4123
Dark: 4047
Physics: 4027
Realistic: 4023
Minimalist: 4013
Free to Play: 4013
Cartoony: 4007
Psychological Horror: 3935
Choices

In [14]:
# one hot encoding first 100 tags, genres top 11, categories top 31
tags_to_encode = list(tags.keys())[:100]
genres_to_encode = list(genres.keys())[:11]
categories_to_encode = list(categories.keys())[:31]

# One-hot encoding the top tags, genres, and categories, no prefix since they all describe the same thing, and if overlap will still be the same
for tag in tags_to_encode:
    df[tag] = df['Tags'].apply(lambda x: 1 if tag in x else 0)
for genre in genres_to_encode:
    df[genre] = df['Genres'].apply(lambda x: 1 if genre in x else 0)
for category in categories_to_encode:
    df[category] = df['Categories'].apply(lambda x: 1 if category in x else 0)

df.drop(columns=['Tags', 'Genres', 'Categories'], inplace=True, errors='ignore') # Drop the original columns after one-hot encoding

C:\Users\jyeal\AppData\Local\Temp\ipykernel_17204\3842760892.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[tag] = df['Tags'].apply(lambda x: 1 if tag in x else 0)
C:\Users\jyeal\AppData\Local\Temp\ipykernel_17204\3842760892.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[tag] = df['Tags'].apply(lambda x: 1 if tag in x else 0)
C:\Users\jyeal\AppData\Local\Temp\ipykernel_17204\3842760892.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which

In [15]:
for col in ['Developers', 'Publishers']:
    if col in df.columns:
        # Calculate frequency of each column
        frequency_map = df[col].value_counts()

        # Map the frequency to the column
        df = df.copy() # Just to fix the de-fragmented frame warning
        df[f'{col}_freq'] = df[col].map(frequency_map)

        display(df[[f'{col}_freq']].head())

,Developers_freq
0,1
1,4
3,1
4,1
5,1


,Publishers_freq
0,1
1,5
3,1
4,1
5,1


In [16]:
df['About Length'] = df['About the game'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)
df['About Length'].describe()

count    66272.000000
mean       215.138671
std        161.855572
min          1.000000
25%        113.000000
50%        177.000000
75%        271.000000
max       4355.000000
Name: About Length, dtype: float64

In [17]:
# Final DataFrame Overview
print(f"Final DataFrame shape: {df.shape}")
print(df.dtypes.value_counts())

final_missing = df.isnull().sum().sum()
print(f"Total missing values in final DataFrame: {final_missing}")

df.to_csv('games_Cleaned.csv', index=False, encoding='utf-8') # Save the cleaned DataFrame to a new CSV file


Final DataFrame shape: (66272, 166)
int64      147
float64     11
object       4
int32        4
dtype: int64
Total missing values in final DataFrame: 0


PermissionError: [Errno 13] Permission denied: 'games_Cleaned.csv'